In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Reshape, Flatten, Lambda

In [ ]:
df = pd.read_csv('final.csv')

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df.head()

,Sentence,ID,Target
0,You may want to drop the col1 column.,4,"df.drop(columns = 'col1',inplace = True)"
1,The column should be dropped.,4,"df.drop(columns = 'col1',inplace = True)"
2,The col 1 column must be dropped.,4,"df.drop(columns = 'col1',inplace = True)"
3,"The dataframe was sorted by col1, col2 and col3.",3,"df.sort_values(by=[‘col1’, ‘col2’, ‘col3’], in..."
4,Send me a list of col 1 and col 2.,5,"new_df=df.loc[:, ['col1','col2']]"


In [ ]:
df.shape

(984, 3)

In [ ]:
df['Target']

0               df.drop(columns = 'col1',inplace = True)
1               df.drop(columns = 'col1',inplace = True)
2               df.drop(columns = 'col1',inplace = True)
3      df.sort_values(by=[‘col1’, ‘col2’, ‘col3’], in...
4                      new_df=df.loc[:, ['col1','col2']]
                             ...                        
979                    new_df=df.loc[:, ['col1','col2']]
980                            df['col1'].value_counts()
981                            df['col1'].value_counts()
982                            df['col1'].value_counts()
983            df.sort_values(by=[‘col1’], inplace=True)
Name: Target, Length: 984, dtype: object

In [ ]:
# Initialize the tokenizer with filters set to none so it doesn't remove any characters
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')

# Fit the tokenizer on the input and target sequences
tokenizer.fit_on_texts(df['Patterns'])
tokenizer.fit_on_texts(df['Target'])

# Convert the input and target sequences to numerical sequences
input_sequences = tokenizer.texts_to_sequences(df['Patterns'])
target_sequences = tokenizer.texts_to_sequences(df['Target'])

# Determine the length of the longest input sequence and target sequence
max_input_length = max(len(seq) for seq in input_sequences)
max_target_length = max(len(seq) for seq in target_sequences)

# Pad the input and target sequences to the maximum length
padded_input_sequences = tf.keras.preprocessing.sequence.pad_sequences(input_sequences, 
                                                                       maxlen=max_input_length, 
                                                                       padding='post')
padded_target_sequences = tf.keras.preprocessing.sequence.pad_sequences(target_sequences, 
                                                                        maxlen=max_target_length, 
                                                                        padding='post')



In [ ]:
import pickle

# Save the tokenizer
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [ ]:
# Set the percentage of the data to use for validation
test_percent = 0.2

# Determine the number of samples to use for validation
num_test_samples = int(len(df) * test_percent)

# Split the data into training and validation sets
train_input = padded_input_sequences[:-num_test_samples]
train_target = padded_target_sequences[:-num_test_samples]
test_input = padded_input_sequences[-num_test_samples:]
test_target = padded_target_sequences[-num_test_samples:]


In [ ]:
# Define the input shape
encoder_inputs = Input(shape=(max_input_length,))
decoder_inputs = Input(shape=(max_target_length-1,))

# Define the embedding layer
embedding_layer = Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=256)

# Apply the embedding layer to the inputs
encoder_inputs_embedded = embedding_layer(encoder_inputs)
decoder_inputs_embedded = embedding_layer(decoder_inputs)

# Define the encoder LSTM
encoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs_embedded)

# Define the decoder LSTM
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs_embedded, initial_state=[state_h, state_c])

# Define the output layer
output_layer = Dense(len(tokenizer.word_index)+1, activation='softmax')
outputs = output_layer(decoder_outputs)



In [ ]:
# Define the model
input_sequence_length = max_input_length
output_sequence_length = max_target_length - 1
input_vocab_size = len(tokenizer.word_index) + 1
output_vocab_size = len(tokenizer.word_index) + 1
hidden_size = 256


In [ ]:
from keras.optimizers import Adam
# Define the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#  Compile the model
learning_rate = 0.001
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#  Train the model
model.fit([train_input, train_target[:, :-1]], train_target[:, 1:], batch_size=128, epochs=30, validation_split=0.0)


Epoch 1/30


/Users/rithvik17/opt/anaconda3/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


7/7 [==============================] - 5s 136ms/step - loss: 8.1065 - accuracy: 0.1434
Epoch 2/30
7/7 [==============================] - 1s 137ms/step - loss: 2.6979 - accuracy: 0.6675
Epoch 3/30
7/7 [==============================] - 1s 133ms/step - loss: 2.0354 - accuracy: 0.6669
Epoch 4/30
7/7 [==============================] - 1s 134ms/step - loss: 1.8472 - accuracy: 0.6685
Epoch 5/30
7/7 [==============================] - 1s 132ms/step - loss: 1.5704 - accuracy: 0.7167
Epoch 6/30
7/7 [==============================] - 1s 135ms/step - loss: 1.2337 - accuracy: 0.7751
Epoch 7/30
7/7 [==============================] - 1s 134ms/step - loss: 0.9009 - accuracy: 0.8623
Epoch 8/30
7/7 [==============================] - 1s 134ms/step - loss: 0.8076 - accuracy: 0.8560
Epoch 9/30
7/7 [==============================] - 1s 137ms/step - loss: 0.6121 - accuracy: 0.9210
Epoch 10/30
7/7 [==============================] - 1s 134ms/step - loss: 0.9437 - accuracy: 0.8445
Epoch 11/30
7/7 [=============

In [ ]:
# Test the model
test_loss, test_acc = model.evaluate([test_input, test_target[:, :-1]], test_target[:, 1:])
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

7/7 [==============================] - 1s 27ms/step - loss: 0.3423 - accuracy: 0.9452
Test loss: 0.3422703146934509
Test accuracy: 0.9451530575752258


In [ ]:
test_predictions = model.predict([test_input, test_target[:, :-1]])

test_predictions = np.argmax(test_predictions, axis=-1)

true_targets = test_target[:, 1:].flatten()
predicted_targets = test_predictions.flatten()

from sklearn.metrics import classification_report
report = classification_report(true_targets, predicted_targets, zero_division=1)
print(report)


7/7 [==============================] - 1s 39ms/step
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       506
           3       1.00      1.00      1.00        86
          16       1.00      1.00      1.00        43
          17       1.00      0.00      0.00        43
          19       1.00      1.00      1.00        41
          21       1.00      1.00      1.00        43
          54       1.00      1.00      1.00         7
          55       1.00      1.00      1.00         7
          56       1.00      1.00      1.00         8

    accuracy                           0.95       784
   macro avg       0.99      0.89      0.88       784
weighted avg       0.95      0.95      0.92       784



In [ ]:
model.save('seq2seq_model.h5')